# 02 — EVT — GEV (MLE) Batch Fitting Across All Cities (Ecuador)

**Updated:** 2026-02-23  
**Goal:** Fit **GEV** to **block maxima** for each city and variable, using **MLE** (scipy).

This notebook is designed for your repo structure:

```
/content/drive/MyDrive/extreme-climate-forecasting/
├── data/_tmp_evt_city_sorted_final/<CITY>.parquet
└── results/
```

## Outputs
- `results/gev_mle_summary.csv`
- `results/gev_mle_summary.parquet`
- `results/gev_mle_batch_errors.csv` (fit/load errors)
- `results/gev_mle_plot_errors.csv` (plot generation errors)
- `results/mle_<city>_<var>_<rule>/` diagnostic plots *(optional)*

## Key robustness features
- Safe datetime-column detection (handles categorical/object dtypes).
- Safe string operations (`status`).
- Safe sorting even if empty.
- Plot creation is isolated from fitting errors (plot errors logged separately).
- Includes a “rebuild missing plots” utility cell.

---


In [1]:
# Mount Google Drive (Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1) Imports & configuration

In [2]:
from __future__ import annotations

from pathlib import Path
import warnings

import numpy as np
import pandas as pd

from pandas.api.types import is_datetime64_any_dtype, is_numeric_dtype

from scipy.stats import genextreme

import matplotlib.pyplot as plt

from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)

# -----------------------------
# Paths
# -----------------------------
PROJECT_DIR = Path("/content/drive/MyDrive/extreme-climate-forecasting")
DATA_DIR = PROJECT_DIR / "data"
CITY_PARQUET_DIR = DATA_DIR / "_tmp_evt_city_sorted_final"
RESULTS_DIR = PROJECT_DIR / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# -----------------------------
# Variables to fit (EDIT)
# -----------------------------
EVT_VARS = [
    "temp",
    # "temp_max",
    # "precip_1h",
    # "wind_speed",
    # "humidity",
]

# -----------------------------
# Block rule for maxima
# -----------------------------
# 'YE' annual | 'ME' monthly | 'D' daily
BLOCK_RULE = "ME"

# Return periods (years)
RETURN_PERIODS = [10, 20, 50, 100]

# Minimum number of blocks (rule of thumb)
MIN_BLOCKS = 40 if BLOCK_RULE.upper() in ["Y", "YE", "A"] else 50

# Save per-city plots (can be heavy)
SAVE_PLOTS = True

# Optionally limit cities for quick test
MAX_CITIES = None

print("PROJECT_DIR:", PROJECT_DIR)
print("CITY_PARQUET_DIR:", CITY_PARQUET_DIR)
print("RESULTS_DIR:", RESULTS_DIR)
print("BLOCK_RULE:", BLOCK_RULE, "| MIN_BLOCKS:", MIN_BLOCKS)
print("EVT_VARS:", EVT_VARS)


PROJECT_DIR: /content/drive/MyDrive/extreme-climate-forecasting
CITY_PARQUET_DIR: /content/drive/MyDrive/extreme-climate-forecasting/data/_tmp_evt_city_sorted_final
RESULTS_DIR: /content/drive/MyDrive/extreme-climate-forecasting/results
BLOCK_RULE: ME | MIN_BLOCKS: 50
EVT_VARS: ['temp']


## 2) Helper functions

In [3]:
def list_cities_from_folder(city_parquet_dir: Path) -> list[str]:
    if not city_parquet_dir.exists():
        raise FileNotFoundError(f"City parquet folder not found: {city_parquet_dir}")
    cities = sorted([p.stem for p in city_parquet_dir.glob("*.parquet")])
    if len(cities) == 0:
        raise ValueError(f"No parquet files found in: {city_parquet_dir}")
    return cities


def normalize_block_rule(block_rule: str) -> tuple[str, str, int]:
    """Return (normalized_rule, pandas_resample_rule, blocks_per_year)."""
    r = str(block_rule).upper().strip()
    aliases = {"A": "YE", "Y": "YE", "M": "ME"}
    r = aliases.get(r, r)

    supported = {
        "YE": ("YE", 1),
        "ME": ("ME", 12),
        "D": ("D", 365),
    }
    if r not in supported:
        raise ValueError(f"Invalid BLOCK_RULE='{block_rule}'. Use one of {sorted(supported)}")

    pandas_rule, bpy = supported[r]
    return r, pandas_rule, bpy


def detect_datetime_column(df: pd.DataFrame) -> str:
    """Robust datetime-column detection (safe for categorical/object dtypes)."""
    candidates = [
        "fecha_local", "datetime_local", "timestamp_local", "date_local",
        "datetime", "timestamp", "date", "time"
    ]
    for c in candidates:
        if c in df.columns:
            return c

    for c in df.columns:
        try:
            if is_datetime64_any_dtype(df[c]):
                return c
        except Exception:
            # ignore odd extension dtypes (categorical, etc.)
            pass

    raise ValueError(
        "No datetime column found. Add it to candidates or pass dt_col explicitly."
    )


def ensure_datetime(df: pd.DataFrame, dt_col: str) -> pd.DataFrame:
    if not is_datetime64_any_dtype(df[dt_col]):
        df = df.copy()
        df[dt_col] = pd.to_datetime(df[dt_col], errors="coerce")
    return df


def extract_block_maxima(df: pd.DataFrame, dt_col: str, var: str, pandas_rule: str) -> pd.Series:
    return (
        df.dropna(subset=[dt_col, var])
          .set_index(dt_col)[var]
          .resample(pandas_rule)
          .max()
          .dropna()
    )


def fit_gev_mle(x: np.ndarray) -> dict:
    """Fit GEV via scipy.stats.genextreme (MLE). scipy parameter c = -xi."""
    c, mu, sigma = genextreme.fit(x)
    xi = -c

    endpoint = None
    tail_type = "Gumbel"
    if xi < -0.05:
        tail_type = "Weibull"
        endpoint = mu - sigma / xi
    elif xi > 0.05:
        tail_type = "Frechet"

    return {
        "c": float(c),
        "xi": float(xi),
        "loc": float(mu),
        "scale": float(sigma),
        "tail_type": tail_type,
        "endpoint": None if endpoint is None else float(endpoint),
    }


def compute_return_levels(params: dict, blocks_per_year: int, return_periods_years: list[int]) -> dict:
    c = params["c"]
    mu = params["loc"]
    sigma = params["scale"]
    out = {}
    for T in return_periods_years:
        T_blocks = blocks_per_year * float(T)
        rl = genextreme.ppf(1 - 1 / T_blocks, c, loc=mu, scale=sigma)
        out[f"RL{int(T)}"] = float(rl)
    return out


def save_diagnostics_plots(out_dir: Path, city: str, var: str, block_max: pd.Series, params: dict, blocks_per_year: int, block_rule: str):
    out_dir.mkdir(parents=True, exist_ok=True)
    c = float(params["c"])
    mu = float(params["loc"])
    sigma = float(params["scale"])
    xi = float(params.get("xi", -c))

    safe_city = str(city).lower().replace(" ", "_")
    safe_var = str(var).lower().replace(" ", "_")

    # 1) Block maxima time series
    plt.figure(figsize=(10, 4))
    plt.plot(block_max.index, block_max.values, linewidth=1)
    plt.title(f"Block maxima ({block_rule}) — {city} — {var}")
    plt.xlabel("Date")
    plt.ylabel(var)
    plt.tight_layout()
    plt.savefig(out_dir / f"block_maxima_{safe_city}_{safe_var}.png", dpi=200)
    plt.close()

    # 2) QQ plot
    emp_q = np.sort(block_max.values)
    n = len(emp_q)
    p = (np.arange(1, n + 1) - 0.44) / (n + 0.12)
    theo_q = genextreme.ppf(p, c, loc=mu, scale=sigma)

    plt.figure(figsize=(5.5, 5.5))
    plt.scatter(theo_q, emp_q, s=18, alpha=0.8)
    mn = min(theo_q.min(), emp_q.min())
    mx = max(theo_q.max(), emp_q.max())
    plt.plot([mn, mx], [mn, mx], linestyle="--")
    plt.title(f"Q-Q — {city} — {var}\nxi={xi:.3f}, mu={mu:.2f}, sigma={sigma:.2f}")
    plt.xlabel("Theoretical quantiles (GEV)")
    plt.ylabel("Observed quantiles")
    plt.tight_layout()
    plt.savefig(out_dir / f"qq_{safe_city}_{safe_var}.png", dpi=200)
    plt.close()

    # 3) Return level curve (1..~200 years)
    T_curve = np.logspace(0, 2.3, 250)  # 1 .. ~200
    RL_curve = genextreme.ppf(1 - 1/(blocks_per_year * T_curve), c, loc=mu, scale=sigma)

    plt.figure(figsize=(7, 5))
    plt.plot(T_curve, RL_curve, linewidth=2)
    plt.xscale("log")
    plt.title(f"Return level — {city} — {var}")
    plt.xlabel("Return period (years, log)")
    plt.ylabel("Return level")
    plt.tight_layout()
    plt.savefig(out_dir / f"return_level_{safe_city}_{safe_var}.png", dpi=200)
    plt.close()


## 3) Discover cities

In [4]:
BLOCK_RULE, PANDAS_RULE, BLOCKS_PER_YEAR = normalize_block_rule(BLOCK_RULE)

CITIES = list_cities_from_folder(CITY_PARQUET_DIR)
if MAX_CITIES is not None:
    CITIES = CITIES[: int(MAX_CITIES)]

print("Cities found:", len(CITIES))
print("First 10:", CITIES[:10])
print("Resample rule:", PANDAS_RULE, "| Blocks/year:", BLOCKS_PER_YEAR)


Cities found: 16
First 10: ['Ambato', 'Cuenca', 'Esmeraldas', 'Guayaquil', 'Ibarra', 'Lago Agrio', 'Loja', 'Machala', 'Manta', 'Puerto Morona']
Resample rule: ME | Blocks/year: 12


## 4) Batch fit (GEV MLE) with progress bars

In [5]:
results = []
errors = []
plot_errors = []

outer = tqdm(CITIES, desc="Cities", leave=True)
for city in outer:
    city_path = CITY_PARQUET_DIR / f"{city}.parquet"

    # ---------- load ----------
    try:
        df_city = pd.read_parquet(city_path)
    except Exception as e:
        errors.append({"city": city, "var": None, "where": "read_parquet", "error": str(e)})
        continue

    # ---------- datetime column ----------
    try:
        dt_col = detect_datetime_column(df_city)
        df_city = ensure_datetime(df_city, dt_col)
    except Exception as e:
        errors.append({"city": city, "var": None, "where": "datetime_detection", "error": str(e)})
        continue

    # ---------- per-variable loop ----------
    inner = tqdm(EVT_VARS, desc=f"{city}: vars", leave=False)
    for var in inner:
        row = {
            "status": None,
            "city": city,
            "var": var,
            "block_rule": BLOCK_RULE,
            "dt_col": dt_col,
            "n_rows": int(len(df_city)),
            "start_dt": None,
            "end_dt": None,
            "n_blocks": None,
        }

        try:
            if var not in df_city.columns:
                row["status"] = "skip_missing_var"
                results.append(row)
                continue

            if not is_numeric_dtype(df_city[var]):
                row["status"] = "skip_non_numeric_var"
                results.append(row)
                continue

            bm = extract_block_maxima(df_city, dt_col, var, PANDAS_RULE)
            row["n_blocks"] = int(len(bm))
            row["start_dt"] = str(bm.index.min().date()) if len(bm) else None
            row["end_dt"] = str(bm.index.max().date()) if len(bm) else None

            if len(bm) < MIN_BLOCKS:
                row["status"] = f"skip_insufficient_blocks_{len(bm)}"
                results.append(row)
                continue

            params = fit_gev_mle(bm.values.astype(float))
            rls = compute_return_levels(params, BLOCKS_PER_YEAR, RETURN_PERIODS)

            row.update({
                "status": "ok",
                "c": params["c"],
                "xi": params["xi"],
                "loc": params["loc"],
                "scale": params["scale"],
                "tail_type": params["tail_type"],
                "endpoint": params["endpoint"],
            })
            row.update(rls)
            results.append(row)

            # ---------- plots (isolated errors) ----------
            if SAVE_PLOTS:
                out_dir = RESULTS_DIR / f"mle_{str(city).lower().replace(' ','_')}_{str(var).lower()}_{BLOCK_RULE.lower()}"
                try:
                    save_diagnostics_plots(out_dir, city, var, bm, params, BLOCKS_PER_YEAR, BLOCK_RULE)
                except Exception as pe:
                    plot_errors.append({
                        "city": city, "var": var, "where": "save_diagnostics_plots", "error": str(pe)
                    })

        except Exception as e:
            row["status"] = "error"
            results.append(row)
            errors.append({"city": city, "var": var, "where": "fit_loop", "error": str(e)})

summary_df = pd.DataFrame(results)
errors_df = pd.DataFrame(errors)
plot_errors_df = pd.DataFrame(plot_errors)

print("Done.")
print("Summary rows:", len(summary_df))
print("Errors rows:", len(errors_df))
print("Plot errors rows:", len(plot_errors_df))


Cities:   0%|          | 0/16 [00:00<?, ?it/s]

Ambato: vars:   0%|          | 0/1 [00:00<?, ?it/s]

Cuenca: vars:   0%|          | 0/1 [00:00<?, ?it/s]

Esmeraldas: vars:   0%|          | 0/1 [00:00<?, ?it/s]

Guayaquil: vars:   0%|          | 0/1 [00:00<?, ?it/s]

...

Quito: vars:   0%|          | 0/1 [00:00<?, ?it/s]

Santa Cruz Island: vars:   0%|          | 0/1 [00:00<?, ?it/s]

Santo Domingo: vars:   0%|          | 0/1 [00:00<?, ?it/s]

Zamora: vars:   0%|          | 0/1 [00:00<?, ?it/s]

Done.
Summary rows: 16
Errors rows: 0
Plot errors rows: 0


## 5) Save outputs (robust schema + safe sorting)

In [6]:
OUT_SUMMARY_CSV = RESULTS_DIR / "gev_mle_summary.csv"
OUT_SUMMARY_PARQUET = RESULTS_DIR / "gev_mle_summary.parquet"
OUT_ERRORS = RESULTS_DIR / "gev_mle_batch_errors.csv"
OUT_PLOT_ERRORS = RESULTS_DIR / "gev_mle_plot_errors.csv"

EXPECTED_COLS = (
    ["status", "city", "var", "block_rule", "dt_col",
     "n_rows", "start_dt", "end_dt", "n_blocks",
     "c", "xi", "loc", "scale", "tail_type", "endpoint"]
    + [f"RL{T}" for T in RETURN_PERIODS]
)

for col in EXPECTED_COLS:
    if col not in summary_df.columns:
        summary_df[col] = np.nan

summary_df = summary_df[EXPECTED_COLS + [c for c in summary_df.columns if c not in EXPECTED_COLS]]

sort_cols = [c for c in ["status", "city", "var"] if c in summary_df.columns]
if sort_cols:
    summary_df = summary_df.sort_values(sort_cols).reset_index(drop=True)

summary_df.to_csv(OUT_SUMMARY_CSV, index=False)
summary_df.to_parquet(OUT_SUMMARY_PARQUET, index=False)
print("Saved:")
print(" -", OUT_SUMMARY_CSV)
print(" -", OUT_SUMMARY_PARQUET)

if len(errors_df) > 0:
    errors_df.to_csv(OUT_ERRORS, index=False)
    print(" -", OUT_ERRORS)

if len(plot_errors_df) > 0:
    plot_errors_df.to_csv(OUT_PLOT_ERRORS, index=False)
    print(" -", OUT_PLOT_ERRORS)

summary_df.tail()

Saved:
 - /content/drive/MyDrive/extreme-climate-forecasting/results/gev_mle_summary.csv
 - /content/drive/MyDrive/extreme-climate-forecasting/results/gev_mle_summary.parquet


,status,city,var,block_rule,dt_col,n_rows,start_dt,end_dt,n_blocks,c,xi,loc,scale,tail_type,endpoint,RL10,RL20,RL50,RL100
11,ok,Quevedo,temp,ME,fecha_local,402121,1978-12-31,2024-10-31,551,0.176831,-0.176831,28.496177,1.630409,Weibull,37.716340,33.759063,34.216854,34.740977,35.084397
12,ok,Quito,temp,ME,fecha_local,411112,1978-12-31,2024-10-31,551,0.137318,-0.137318,19.629468,1.673382,Weibull,31.815637,25.497234,26.072548,26.752421,27.212385
13,ok,Santa Cruz Island,temp,ME,fecha_local,401496,1978-12-31,2024-10-31,551,0.084459,-0.084459,21.147642,1.793176,Weibull,42.378975,28.203905,29.012289,30.009018,30.712807
14,ok,Santo Domingo,temp,ME,fecha_local,403049,1978-12-31,2024-10-31,551,0.317828,-0.317828,27.166247,1.232858,Weibull,31.045263,30.197103,30.365256,30.537263,30.637760
15,ok,Zamora,temp,ME,fecha_local,401496,1978-12-31,2024-10-31,551,0.380479,-0.380479,31.818877,1.541278,Weibull,35.869769,35.213386,35.365949,35.514415,35.596836


## 6) Diagnostics (OK / skipped / errors)

In [7]:
status_s = summary_df["status"].fillna("").astype(str)

ok = summary_df[summary_df["status"] == "ok"].copy()
skipped = summary_df[status_s.str.startswith("skip_")].copy()
err_rows = summary_df[summary_df["status"] == "error"].copy()

print("OK fits:", len(ok))
print("Skipped:", len(skipped))
print("Errors (in summary):", len(err_rows))
print("Errors (log file):", len(errors_df))
print("Plot errors (log file):", len(plot_errors_df))

if len(ok) > 0:
    display(ok.groupby("city").size().sort_values(ascending=False).head(20))


OK fits: 16
Skipped: 0
Errors (in summary): 0
Errors (log file): 0
Plot errors (log file): 0


,0
city,
Ambato,1
Cuenca,1
Esmeraldas,1
Guayaquil,1
Ibarra,1
Lago Agrio,1
Loja,1
Machala,1
Manta,1


## 7) Check which cities have plot folders (and which are missing)

In [8]:
if len(ok) == 0:
    print("No OK rows, skipping plot-folder check.")
else:
    expected = set(ok["city"].unique())

    have = set()
    for p in RESULTS_DIR.glob("mle_*"):
        name = p.name.replace("mle_", "")
        city_part = name.rsplit("_", 2)[0]  # strip _<var>_<rule>
        have.add(city_part)

    # normalize expected to folder naming convention
    expected_folder_names = set([str(c).lower().replace(" ", "_") for c in expected])

    missing = sorted(expected_folder_names - have)
    print("Expected plot folders:", len(expected_folder_names))
    print("Existing plot folders:", len(have))
    print("Missing plot folders:", len(missing))
    if missing:
        print("Missing (first 30):", missing[:30])


Expected plot folders: 16
Existing plot folders: 16
Missing plot folders: 0


## 8) Rebuild missing plots (utility)
Run this if the summary is complete but some plot folders are missing.

In [9]:
# Rebuild plots only for missing folders
if len(ok) == 0:
    print("No OK fits found.")
else:
    expected = set([str(c).lower().replace(' ','_') for c in ok['city'].unique()])

    have = set()
    for p in RESULTS_DIR.glob("mle_*"):
        name = p.name.replace("mle_", "")
        city_part = name.rsplit("_", 2)[0]
        have.add(city_part)

    missing_cities = sorted(expected - have)
    print("Missing plot folders:", len(missing_cities))

    regen_errors = []
    for city_slug in tqdm(missing_cities, desc="Rebuilding missing plots"):
        # recover the original city name from ok rows
        city_name = ok.loc[ok['city'].astype(str).str.lower().str.replace(' ','_') == city_slug, 'city'].iloc[0]

        for var in EVT_VARS:
            sel = ok[(ok["city"] == city_name) & (ok["var"] == var)]
            if len(sel) == 0:
                continue
            r = sel.iloc[0]

            try:
                df_city = pd.read_parquet(CITY_PARQUET_DIR / f"{city_name}.parquet")
                dt_col = detect_datetime_column(df_city)
                df_city = ensure_datetime(df_city, dt_col)
                bm = extract_block_maxima(df_city, dt_col, var, PANDAS_RULE)

                params = {"c": float(r["c"]), "xi": float(r["xi"]), "loc": float(r["loc"]), "scale": float(r["scale"])}

                out_dir = RESULTS_DIR / f"mle_{city_slug}_{str(var).lower()}_{BLOCK_RULE.lower()}"
                save_diagnostics_plots(out_dir, city_name, var, bm, params, BLOCKS_PER_YEAR, BLOCK_RULE)

            except Exception as e:
                regen_errors.append({"city": city_name, "var": var, "error": str(e)})

    print("Rebuild done. Regen errors:", len(regen_errors))
    if regen_errors:
        display(pd.DataFrame(regen_errors).head(20))


Missing plot folders: 0


Rebuilding missing plots: 0it [00:00, ?it/s]

Rebuild done. Regen errors: 0
